# Starter Notebook

Install and import required libraries

In [1]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, ClassLabel
import pickle

/home/nd2745/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Tokenizer and Preprocess Data

In [2]:
base_model = 'roberta-base'

dataset = load_dataset('ag_news', split='train')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

def preprocess(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized

tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [3]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


## Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [4]:
model = RobertaForSequenceClassification.from_pretrained(
    base_model,
    id2label=id2label)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

## Anything from here on can be modified

In [5]:
# Split the original training set
split_datasets = tokenized_dataset.train_test_split(test_size=640, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

## Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [6]:
# PEFT Config
peft_config = LoraConfig(
    r=21,
    lora_alpha=40,
    lora_dropout=0,
    bias = 'lora_only',
    target_modules = ['query'],
    task_type="SEQ_CLS",
)


In [7]:
peft_model = get_peft_model(model, peft_config)
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
          

In [8]:
# print("Trainable parameters:")
# for name, param in peft_model.named_parameters():
#     if param.requires_grad:
#         print(name)

In [9]:
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 989,956 || all params: 125,629,448 || trainable%: 0.7880


## Training Setup

In [10]:
# To track evaluation accuracy during training
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy
    }

In [13]:
# Setup Training args
output_dir = "results_trial"
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",              # ensure saving
    save_total_limit=1,                 # optional: save last only
    learning_rate=0.001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    optim="adamw_torch",
    report_to=None,
    logging_steps=100,
    disable_tqdm=True
)

def get_trainer(model):
      return  Trainer(
          model=model,
          args=training_args,
          compute_metrics=compute_metrics,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

/home/nd2745/.local/lib/python3.9/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Start Training

In [14]:
peft_lora_finetuning_trainer = get_trainer(peft_model)

result = peft_lora_finetuning_trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


{'loss': 0.6088, 'grad_norm': 0.5143557190895081, 'learning_rate': 0.0009986595174262735, 'epoch': 0.013404825737265416}
{'loss': 0.3564, 'grad_norm': 2.1240227222442627, 'learning_rate': 0.0009973190348525468, 'epoch': 0.02680965147453083}
{'loss': 0.3757, 'grad_norm': 1.817781686782837, 'learning_rate': 0.0009959785522788203, 'epoch': 0.040214477211796246}
{'loss': 0.3041, 'grad_norm': 2.3164684772491455, 'learning_rate': 0.0009946380697050939, 'epoch': 0.05361930294906166}
{'loss': 0.3366, 'grad_norm': 1.362421989440918, 'learning_rate': 0.0009932975871313674, 'epoch': 0.06702412868632708}
{'loss': 0.3102, 'grad_norm': 2.737419366836548, 'learning_rate': 0.0009919571045576407, 'epoch': 0.08042895442359249}
{'loss': 0.3163, 'grad_norm': 0.7768217325210571, 'learning_rate': 0.0009906166219839142, 'epoch': 0.0938337801608579}
{'loss': 0.3309, 'grad_norm': 1.783843994140625, 'learning_rate': 0.0009892761394101877, 'epoch': 0.10723860589812333}
{'loss': 0.3123, 'grad_norm': 3.79311728477

## Evaluate Finetuned Model


### Performing Inference on Custom Input
Uncomment following functions for running inference on custom inputs

In [15]:
def classify(model, tokenizer, text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
    output = model(**inputs)

    prediction = output.logits.argmax(dim=-1).item()

    print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
    return id2label[prediction]

In [16]:
classify( peft_model, tokenizer, "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( peft_model, tokenizer, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")


 Class: 0, Label: World, Text: Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...

 Class: 2, Label: Business, Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindlinand of ultra-cynics, are seeing green again.


'Business'

### Run Inference on eval_dataset

In [17]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=8, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    if labelled:
        metric = evaluate.load('accuracy')

    # Loop over the DataLoader
    for batch in tqdm(eval_dataloader):
        # Move each tensor in the batch to the device
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions.append(predictions.cpu())

        if labelled:
            # Expecting that labels are provided under the "labels" key.
            references = batch["labels"]
            metric.add_batch(
                predictions=predictions.cpu().numpy(),
                references=references.cpu().numpy()
            )

    # Concatenate predictions from all batches
    all_predictions = torch.cat(all_predictions, dim=0)

    if labelled:
        eval_metric = metric.compute()
        print("Evaluation Metric:", eval_metric)
        return eval_metric, all_predictions
    else:
        return all_predictions

In [18]:
# Check evaluation accuracy
_, _ = evaluate_model(peft_model, eval_dataset, True, 8, data_collator)

100%|██████████| 80/80 [00:06<00:00, 12.40it/s]

Evaluation Metric: {'accuracy': 0.9109375}


In [19]:
print(_)

tensor([0, 1, 0, 3, 0, 3, 0, 3, 3, 2, 1, 2, 3, 0, 2, 1, 1, 0, 3, 2, 3, 2, 2, 2,
        0, 1, 3, 2, 1, 1, 3, 1, 0, 1, 2, 0, 0, 0, 2, 3, 0, 3, 1, 1, 2, 3, 0, 0,
        0, 1, 3, 1, 0, 3, 1, 1, 3, 0, 3, 2, 3, 3, 0, 0, 1, 3, 1, 1, 1, 2, 0, 3,
        2, 3, 2, 1, 1, 3, 1, 2, 0, 3, 2, 0, 1, 2, 2, 3, 1, 0, 1, 0, 3, 3, 2, 1,
        2, 1, 2, 1, 2, 2, 2, 2, 1, 3, 0, 3, 1, 2, 3, 3, 0, 1, 2, 0, 1, 1, 2, 1,
        0, 0, 3, 2, 3, 3, 3, 2, 1, 2, 3, 3, 2, 0, 3, 0, 0, 0, 1, 0, 3, 1, 0, 3,
        0, 3, 0, 3, 0, 2, 1, 0, 0, 3, 3, 3, 2, 1, 3, 0, 0, 2, 0, 1, 3, 3, 3, 2,
        3, 1, 1, 3, 2, 2, 0, 0, 2, 2, 3, 0, 1, 2, 1, 0, 2, 0, 1, 3, 2, 3, 0, 1,
        2, 0, 0, 3, 3, 3, 1, 2, 1, 2, 2, 1, 3, 2, 2, 1, 0, 3, 3, 1, 0, 0, 0, 0,
        0, 2, 0, 3, 2, 2, 0, 2, 3, 2, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 1, 1, 2, 1,
        3, 1, 1, 1, 2, 2, 3, 0, 1, 0, 2, 0, 1, 3, 1, 2, 2, 1, 3, 2, 0, 0, 0, 2,
        1, 1, 0, 1, 3, 2, 1, 2, 2, 1, 0, 2, 1, 3, 2, 3, 0, 2, 2, 2, 2, 1, 2, 0,
        1, 1, 3, 3, 1, 3, 0, 2, 1, 3, 0,

### Run Inference on unlabelled dataset

In [20]:
#Load your unlabelled data
unlabelled_dataset = pd.read_pickle("test_unlabelled.pkl")
test_dataset = unlabelled_dataset.map(preprocess, batched=True, remove_columns=["text"])
unlabelled_dataset

Map: 100%|██████████| 8000/8000 [00:05<00:00, 1376.78 examples/s]


Dataset({
    features: ['text'],
    num_rows: 8000
})

In [21]:
# Run inference and save predictions
preds = evaluate_model(peft_model, test_dataset, False, 8, data_collator)
df_output = pd.DataFrame({
    'ID': range(len(preds)),
    'Label': preds.numpy()  # or preds.tolist()
})
df_output.to_csv(os.path.join(output_dir,"inference_output_1.csv"), index=False)
print("Inference complete. Predictions saved to inference_output.csv")

100%|██████████| 1000/1000 [01:02<00:00, 16.03it/s]

Inference complete. Predictions saved to inference_output.csv
